# Langkah 0 - Persiapan

In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Langkah 1 - Memuat Data

In [8]:
df = pd.read_csv('dataset/wbc.csv')
df.head()


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


# Langkah 2 - Pisahkan variabel yang tidak digunakan

In [9]:
# Variabel yang tidak digunakan
variabel_tidak_digunakan = ["id", "Unnamed: 32"]

# Target
variabel_target = "diagnosis"

# Variabel fitur
variabel_fitur = [col for col in df.columns if col not in variabel_tidak_digunakan + [variabel_target]]

print("Variabel Tidak Digunakan:", variabel_tidak_digunakan)
print("Variabel Fitur:", variabel_fitur[:5], "... total:", len(variabel_fitur))


Variabel Tidak Digunakan: ['id', 'Unnamed: 32']
Variabel Fitur: ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean'] ... total: 30


# Langkah 3 - Encoding

In [10]:
encoder = LabelEncoder()
df["diagnosis_encoded"] = encoder.fit_transform(df["diagnosis"])

# Mapping hasil encoding
dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))


{'B': np.int64(0), 'M': np.int64(1)}

# Langkah 4 - Standarisasi

In [11]:
scaler = StandardScaler()
df[variabel_fitur] = scaler.fit_transform(df[variabel_fitur])

df.head()


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32,diagnosis_encoded
0,842302,M,1.097064,-2.073335,1.269934,0.984375,1.568466,3.283515,2.652874,2.532475,...,2.303601,2.001237,1.307686,2.616665,2.109526,2.296076,2.750622,1.937015,NaN,1
1,842517,M,1.829821,-0.353632,1.685955,1.908708,-0.826962,-0.487072,-0.023846,0.548144,...,1.535126,1.890489,-0.375612,-0.430444,-0.146749,1.087084,-0.243890,0.281190,NaN,1
2,84300903,M,1.579888,0.456187,1.566503,1.558884,0.942210,1.052926,1.363478,2.037231,...,1.347475,1.456285,0.527407,1.082932,0.854974,1.955000,1.152255,0.201391,NaN,1
3,84348301,M,-0.768909,0.253732,-0.592687,-0.764464,3.283553,3.402909,1.915897,1.451707,...,-0.249939,-0.550021,3.394275,3.893397,1.989588,2.175786,6.046041,4.935010,NaN,1
4,84358402,M,1.750297,-1.151816,1.776573,1.826229,0.280372,0.539340,1.371011,1.428493,...,1.338539,1.220724,0.220556,-0.313395,0.613179,0.729259,-0.868353,-0.397100,NaN,1


# Langkah 5 - Split Data untuk membuat data latih dan uji dengan rasio 80:20

In [12]:
from sklearn.model_selection import train_test_split

# Pertama, split data menjadi train (80%) dan temp (20%)
df_train, df_temp = train_test_split(
    df,
    test_size=0.2,
    random_state=0,
    stratify=df['diagnosis_encoded']
)

# Kedua, split temp menjadi validation (10%) dan test (10%)
# Karena temp = 20%, maka test_size=0.5 → masing-masing 10% dari total
df_val, df_test = train_test_split(
    df_temp,
    test_size=0.5,
    random_state=0,
    stratify=df_temp['diagnosis_encoded']
)

# Cek jumlah label di masing-masing subset
print("Jumlah label data asli:\n", df['diagnosis_encoded'].value_counts())
print("\nJumlah label data train:\n", df_train['diagnosis_encoded'].value_counts())
print("\nJumlah label data validation:\n", df_val['diagnosis_encoded'].value_counts())
print("\nJumlah label data test:\n", df_test['diagnosis_encoded'].value_counts())


Jumlah label data asli:
 diagnosis_encoded
0    357
1    212
Name: count, dtype: int64

Jumlah label data train:
 diagnosis_encoded
0    285
1    170
Name: count, dtype: int64

Jumlah label data validation:
 diagnosis_encoded
0    36
1    21
Name: count, dtype: int64

Jumlah label data test:
 diagnosis_encoded
0    36
1    21
Name: count, dtype: int64
